In [1]:
import pandas as pd
import stanza
from nltk.corpus import wordnet as wn
from sklearn.metrics import precision_recall_fscore_support
import nltk

nltk.download('wordnet')
stanza.download('en')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hello\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


2025-02-20 09:16:40 INFO: Downloaded file to C:\Users\hello\stanza_resources\resources.json
2025-02-20 09:16:40 INFO: Downloading default packages for language: en (English) ...
2025-02-20 09:16:41 INFO: File exists: C:\Users\hello\stanza_resources\en\default.zip
2025-02-20 09:16:44 INFO: Finished downloading models and saved to C:\Users\hello\stanza_resources


In [2]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,lemma,depparse')

2025-02-20 09:16:44 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-02-20 09:16:44 INFO: Downloaded file to C:\Users\hello\stanza_resources\resources.json
2025-02-20 09:16:44 WARNING: Language en package default expects mwt, which has been added
2025-02-20 09:16:45 INFO: Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

2025-02-20 09:16:45 INFO: Using device: cpu
2025-02-20 09:16:45 INFO: Loading: tokenize
2025-02-20 09:16:46 INFO: Loading: mwt
2025-02-20 09:16:46 INFO: Loading: pos
2025-02-20 09:16:48 INFO: Loading: lemma
2025-02-20 09:16:48 INFO: Loading: depparse
2025-02-20 09:16:49 INFO: Done loading processors!


In [3]:
data = pd.read_csv("PET_Customized.csv")

In [4]:
def extract_tasks_stanza(sentence):
    doc = nlp(sentence)
    tasks = []

    for sent in doc.sentences:
        actor, action, obj = None, None, None

        for word in sent.words:
            if word.deprel in ("nsubj", "nsubjpass"):
                actor = word.text
            elif word.deprel == "root":
                action = word.lemma
            elif word.deprel in ("dobj", "obl"):
                obj = word.text

        if action and not wn.synsets(action):
            action = None  

        if not actor:
            actor = "System"

        tasks.append({"Actor": actor, "Action": action, "Object": obj})

    return tasks

In [5]:
data["extracted_tasks"] = data["original"].apply(extract_tasks_stanza)
data = data.explode("extracted_tasks")
data[["Actor", "Action", "Object"]] = data["extracted_tasks"].apply(pd.Series)
data.drop(columns=["extracted_tasks"], inplace=True)

In [6]:
data["tokens"] = data["tokens"].apply(eval)
data["ner_tags"] = data["ner_tags"].apply(eval)

In [7]:
task_label_mapping = {
    "B-Actor": "Actor",
    "I-Actor": "Actor",
    "B-Activity": "Action",
    "I-Activity": "Action",
    "B-Activity Data": "Object",
    "I-Activity Data": "Object"
}

In [8]:
def extract_gold_entities(tokens, ner_tags):
    entities = {"Actor": [], "Action": [], "Object": []}
    current_entity, current_tokens = None, []

    for token, tag in zip(tokens, ner_tags):
        if tag.startswith("B-"):
            if current_entity:
                entities[current_entity].append(" ".join(current_tokens))
            current_entity = task_label_mapping.get(tag, None)
            current_tokens = [token] if current_entity else []
        elif tag.startswith("I-") and current_entity:
            current_tokens.append(token)
        else:
            if current_entity:
                entities[current_entity].append(" ".join(current_tokens))
            current_entity, current_tokens = None, []

    if current_entity:
        entities[current_entity].append(" ".join(current_tokens))

    return entities

In [9]:
def evaluate_extraction(row):
    tokens, gold_labels = row["tokens"], row["ner_tags"]

    if not any(tag.startswith(("B-", "I-")) for tag in gold_labels):
        return [], []

    gold_entities = extract_gold_entities(tokens, gold_labels)
    predicted_tasks = extract_tasks_stanza(" ".join(tokens))

    predicted_labels = ["O"] * len(tokens)

    for task in predicted_tasks:
        for i, token in enumerate(tokens):
            if task["Actor"] and token in task["Actor"].split():
                predicted_labels[i] = "B-Actor" if i == 0 else "I-Actor"
            elif task["Action"] and token in task["Action"].split():
                predicted_labels[i] = "B-Activity" if i == 0 else "I-Activity"
            elif task["Object"] and token in task["Object"].split():
                predicted_labels[i] = "B-Activity Data" if i == 0 else "I-Activity Data"

    return predicted_labels, gold_labels

In [10]:
evaluated_data = data.apply(evaluate_extraction, axis=1)
evaluated_data = evaluated_data.dropna()

data["predicted_ner"], data["gold_ner"] = zip(*evaluated_data)

all_predicted = [label for sublist in data["predicted_ner"] for label in sublist]
all_gold = [label for sublist in data["gold_ner"] for label in sublist]

precision, recall, f1, _ = precision_recall_fscore_support(all_gold, all_predicted, average="weighted")

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Precision: 0.3078
Recall: 0.4009
F1-score: 0.2671


C:\Users\hello\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
